# Setup

## Imports

In [ ]:
import os
import string
import pickle
import openai
from IPython.display import clear_output

## API keys

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORG_KEY = os.getenv("OPENAI_ORG_KEY")

## Settings

In [ ]:
model = "gpt-4"
temperature = 1.0
out_dir_name = "output"

## Preamble

In [ ]:
sys_prompt = []

In [ ]:
example_interactions = []

In [ ]:
memory = []
memory += sys_prompt
memory += example_interactions

# Utility functions

In [ ]:
def pretty_parse(memory_: list)-> str: 
    out_string = ""
    for turn in memory_:
        out_string += f"{turn['role'].capitalize()}: {turn['content']}\n"
    return out_string

In [ ]:
def prompt_LLM(memory_: list) -> str:
    raw_response_ = openai.ChatCompletion.create(
        model=model,
        messages=memory_,
        temperature=temperature
        )
    response_ = raw_response_["choices"][0]["message"]["content"]
    return response_

# Run inference

In [ ]:
prompt = "A cube with edge length 1 meter." # Type out the user prompt here

In [ ]:
memory += [{"role": "user", "content": prompt}]
response = prompt_LLM(memory)
memory += [{"role": "assistant", "content": response}]

print(pretty_parse(memory))

# Run interactive inference

In [ ]:
stop = False
stop_string = "__STOP__"
while not stop:
    # Get prompt
    prompt = input("User: ")
    
    # Check stop
    if prompt == stop_string:
        stop = True
        continue

    memory += [{"role": "user", "content": prompt}]

    # show `memory`
    clear_output(wait=True)
    print(pretty_parse(memory))

    # prompt LLM
    response = prompt_LLM(memory)
    memory += [{"role": "assistant", "content": response}]

    # show `memory`
    clear_output(wait=True)
    print(pretty_parse(memory))

    

# Save memory to disk

In [ ]:
prompt = "Summarize the topic of our conversation in four words or less."
memory += [{"role": "user", "content": prompt}]
response = prompt_LLM(memory)

file_name = response.translate(str.maketrans('', '', string.punctuation))
file_name = file_name.replace(' ', '_')
file_name = out_dir_name + "/" + file_name
os.makedirs(out_dir_name, exist_ok=True)
with open(f"{file_name}.p", "wb") as fp:
    pickle.dump(memory[:-1], fp)

# Load memory from disk

In [ ]:
file_name = "Meaning_of_life"

In [ ]:
file_name = out_dir_name + "/" + file_name
with open(f"{file_name}.p", "rb") as fp:
    memory = pickle.load(fp)